In [1]:
%pip install opencv-python

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd 
import numpy as np
import cv2

In [3]:
#set_of_video = ['IS1000a','IS1001a','IS1001b','IS1001c','IS1003b','IS1003d','IS1006b','IS1008a',
#                'IS1008b','IS1008c','IS1008d']

In [7]:

annot_data = pd.read_csv('dome_annotations_M1.csv')
dome_dataset = pd.read_csv('dome_dataset_M1.csv')
dome_dataset
annot_data = annot_data.join(dome_dataset['FMD'])
annot_data = annot_data.join(dome_dataset['FLD'])
annot_data = annot_data.drop(annot_data[annot_data.FMD == 0].index)


In [ ]:
set_of_video = sorted(list(set(annot_data['name'].values)))
set_of_video


In [ ]:
for name_video_folder in set_of_video:
  vidcap = cv2.VideoCapture('amicorpus/' + name_video_folder + '/video/IS1000a.Closeup1.avi')
success,image = vidcap.read()
count = 0
while success:
  cv2.imwrite("test/frame%d.jpg" % count, image)     # save frame as JPEG file      
  success,image = vidcap.read()
  count += 1

In [ ]:
for file_path in set_of_classes:
    list_of_files = sorted(glob.glob('./CREMA-D/VideoFlash/*' + file_path + '*.flv'))
    count = 0
    for video_path in list_of_files:
        order_var = re.findall(r'\d{1,5}',video_path)[0]
        cap = cv2.VideoCapture(video_path)
        success,image = cap.read()
        while success:
            #print("CREMA-D/ImageClass/" + file_path + "/" + order_var + "_frame%d.jpg" % count)
            cv2.imwrite("CREMA-D/ImageClass/" + file_path + "/" + order_var + "_frame%d.jpg" % count, image)
            success,image = cap.read()
            count += 1